In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashion/fashion-mnist_test.csv
/kaggle/input/fashion/fashion-mnist_train.csv


In [61]:
train=pd.read_csv('/kaggle/input/fashion/fashion-mnist_train.csv')
test=pd.read_csv('/kaggle/input/fashion/fashion-mnist_test.csv')

test

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,32,23,14,20,0,0,1,0,0,0
9996,6,0,0,0,0,0,0,0,0,0,...,0,0,0,2,52,23,28,0,0,0
9997,8,0,0,0,0,0,0,0,0,0,...,175,172,172,182,199,222,42,0,1,0
9998,8,0,1,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [62]:
test=np.array(test)
train=np.array(train)
train


array([[2, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [8, 0, 0, ..., 0, 0, 0],
       [8, 0, 0, ..., 0, 0, 0],
       [7, 0, 0, ..., 0, 0, 0]])

In [63]:
m, n = train.shape #m: rows, n:no. of features+1 cuz label column
#np.random.shuffle(train)
j,k=test.shape

In [64]:
data_dev = train[0:1000].T #each column becomes an example instead of each row
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

In [65]:
data_test=test.T
Y_test=data_test[0]
X_test=data_test[1:k]
X_test=X_test / 255

In [66]:
data_train = train[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [67]:
Y_train


array([8, 0, 6, ..., 8, 8, 7])

In [68]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [69]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.16, 500)

Iteration:  0
[8 3 7 ... 8 3 8] [8 0 6 ... 8 8 7]
0.09901694915254237
Iteration:  10
[8 4 4 ... 8 2 8] [8 0 6 ... 8 8 7]
0.331135593220339
Iteration:  20
[8 2 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.4250508474576271
Iteration:  30
[8 2 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.509542372881356
Iteration:  40
[7 4 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.546271186440678
Iteration:  50
[8 4 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.5412033898305085
Iteration:  60
[7 4 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.562677966101695
Iteration:  70
[7 6 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.5789491525423729
Iteration:  80
[7 6 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.5945593220338983
Iteration:  90
[7 6 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.607457627118644
Iteration:  100
[7 6 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.6190847457627119
Iteration:  110
[7 6 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.6293220338983051
Iteration:  120
[7 6 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.6389152542372881
Iteration:  130
[7 6 0 ... 8 8 7] [8 0 6 ... 8 8 7]
0.647542372881356
Iteration:  140
[7 6 0

In [70]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions



In [71]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[8 9 6 0 3 4 4 7 4 8 0 1 9 6 2 0 9 3 3 3 8 7 4 4 3 4 4 6 7 1 5 0 5 4 6 7 3
 4 2 1 4 0 9 0 0 5 2 7 7 2 5 4 2 4 1 4 9 8 0 4 5 5 6 3 5 8 5 9 8 1 6 8 1 3
 2 8 3 4 4 5 3 2 6 8 1 4 7 3 4 4 6 5 0 1 7 3 7 8 4 3 8 5 0 5 3 0 8 5 4 1 0
 7 4 1 9 7 6 9 3 3 2 6 0 4 3 0 1 5 9 7 7 0 3 4 7 2 6 1 3 1 0 2 3 0 3 4 2 8
 9 3 7 4 6 8 8 0 7 5 4 1 9 5 3 7 0 8 5 5 7 7 0 9 6 6 0 4 1 9 6 7 2 3 0 0 8
 3 5 4 3 0 6 1 7 7 5 7 8 8 6 4 3 9 7 6 5 3 2 7 0 9 6 9 4 1 2 7 3 8 3 1 3 0
 0 6 4 2 8 7 2 8 6 2 6 2 8 7 0 1 4 9 5 1 5 0 7 7 7 2 3 8 6 5 5 9 3 6 9 3 1
 3 6 9 1 6 9 9 4 4 3 3 1 4 9 3 7 4 6 7 6 1 1 8 4 4 1 0 3 1 5 5 8 4 7 2 1 8
 2 5 5 6 3 8 9 9 3 9 0 3 4 6 1 5 8 2 7 3 0 3 9 7 6 3 7 3 6 0 6 4 5 8 2 9 8
 6 8 5 3 2 7 3 0 2 3 6 4 4 6 4 0 8 8 2 1 9 4 5 6 6 7 6 2 7 8 7 4 9 3 2 2 5
 4 1 1 3 5 0 3 9 6 7 7 3 5 8 9 1 0 9 0 4 5 2 3 9 6 6 7 9 3 5 0 0 4 7 4 0 2
 3 2 0 3 5 2 6 9 0 3 0 3 3 1 6 8 4 9 9 0 6 3 7 7 4 8 9 5 2 2 4 9 0 3 0 6 5
 6 9 7 8 7 5 0 4 8 0 3 0 8 0 0 9 2 4 1 0 3 7 0 3 9 9 8 7 0 5 0 0 9 8 3 3 7
 3 8 0 6 6 0 6 2 1 9 9 7 

0.772

In [72]:
test_predictions = make_predictions(X_test, W1, b1, W2, b2)
get_accuracy(test_predictions, Y_test)

[0 1 2 ... 8 6 2] [0 1 2 ... 8 8 1]


0.7739